In [2]:
%pip install rdkit


   ---------------------------------------- 0.0/22.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/22.8 MB ? eta -:--:--
    --------------------------------------- 0.5/22.8 MB 2.4 MB/s eta 0:00:10
   - -------------------------------------- 1.0/22.8 MB 2.4 MB/s eta 0:00:10
   -- ------------------------------------- 1.6/22.8 MB 2.3 MB/s eta 0:00:10
   --- ------------------------------------ 2.1/22.8 MB 2.2 MB/s eta 0:00:10
   ---- ----------------------------------- 2.4/22.8 MB 2.2 MB/s eta 0:00:10
   ----- ---------------------------------- 2.9/22.8 MB 2.2 MB/s eta 0:00:09
   ----- ---------------------------------- 3.4/22.8 MB 2.3 MB/s eta 0:00:09
   ------ --------------------------------- 3.9/22.8 MB 2.3 MB/s eta 0:00:09
   ------- -------------------------------- 4.5/22.8 MB 2.3 MB/s eta 0:00:08
   -------- ------------------------------- 5.0/22.8 MB 2.3 MB/s eta 0:00:08
   --------- ------------------------------ 5.5/22.8 MB 2.3 MB/s eta 0:00:08
   ----------


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

df = pd.read_csv("../data/raw/data.csv")

print(df.isna().sum())

# Split into train/test
X = df.drop("flashpoint", axis=1)

y = df["flashpoint"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df.head()

compound          0
flashpoint        0
pure substance    0
smiles            0
source            0
is_silicon        0
is_metallic       0
is_tin            0
is_acid           0
data type         0
dtype: int64


,compound,flashpoint,pure substance,smiles,source,is_silicon,is_metallic,is_tin,is_acid,data type
0,1-aminopropan-2-ol,350.15,1,CC(CN)O,pubchem,0,0,0,0,randonly selected as training point or test point
1,"1-chloro-2,4-dinitrobenzene",467.15,1,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,pubchem,0,0,0,0,randonly selected as training point or test point
2,"1,2-dichloroethane",286.15,1,C(CCl)Cl,pubchem,0,0,0,0,randonly selected as training point or test point
3,"1,2,4-trichlorobenzene",383.15,1,C1=CC(=C(C=C1Cl)Cl)Cl,pubchem,0,0,0,0,randonly selected as training point or test point
4,2-chloroacetaldehyde,361.15,1,C(C=O)Cl,pubchem,0,0,0,0,randonly selected as training point or test point


In [4]:
print(df['data type'].value_counts())
print(df['smiles'].value_counts())

data type
randonly selected as training point or test point    13672
training data                                          582
Non-DIPPR data                                         289
test data                                              153
Name: count, dtype: int64
smiles
Cc1ccccc1                     12
CCCCCCCC                      12
CC(C)CCO                      11
CCCCCCC                       11
CCCCC                         11
                              ..
Nc1ccc(Oc2ccc(N)cc2)cc1        1
CC(N)=O                        1
Nc1ccccn1                      1
CC(C)(C)c1cccc(C(C)(C)C)n1     1
C(CCl)O                        1
Name: count, Length: 10230, dtype: int64


In [ ]:
# Update train-test split with processed features

# Replace original X with processed features

y = df["flashpoint"]

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save processed datasets
X_train.to_csv("../data/processed/X_train.csv", index=False)
y_train.to_csv("../data/processed/y_train.csv", index=False)
X_test.to_csv("../data/processed/X_test.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)

print("Processed datasets saved successfully.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")